# Задача классификации графов, предсказание свойств горения молекул углеводорода
## Постановка задачи: предсказать один из индикаторов качества горения – производное цетановое число (DCN) для оксигенированных углеводородов по структуре молекул.

In [ ]:
import os
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
import torch
import torch_geometric.transforms as T
from torch_geometric.utils import to_dense_adj

from stable_gnn.pipelines.graph_classification_pipeline import TrainModelGC, TrainModelOptunaGC
from stable_gnn.graph import Graph
from stable_gnn.explain import Explain

# Загрузка датасета, состоящего из молекул.
 Атрибуты вершин: относится ли атом к определенному типу (например C, N, S и т.д.), степень вершин-атомов, формальный заряд атома, тип гибридизации, является ли атом частью кольца, является ли атом частью ароматического соединения, нормированная атомная масса

In [ ]:
root = '../data_validation/'
name='fuel'
dataset = Graph(root=root + str(name), name=name, transform=T.NormalizeFeatures(),adjust_flag=False)
len(dataset)

## Решаем задачу предсказания связей, пользуясь подготовленным пайплайном в stable_gnn.pipelines.train_model_pipeline
Задаем различные конфигурации включения экстраполяции и самостоятельного обучения

In [ ]:
results = pd.DataFrame(columns=['extrapolate_flag', 'ssl_flag','test accuracy'])

In [ ]:
conv = "GAT"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ssl_flag = False
extrapolate_flag = False

    #######

optuna_training = TrainModelOptunaGC(
        data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
        )

best_values = optuna_training.run(number_of_trials=50)

model_training = TrainModelGC(
            data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
)

model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(best_values)
print(test_acc_mi)
results=results.append(pd.Series([extrapolate_flag,ssl_flag,test_acc_mi],index=results.columns), ignore_index=True)

|## Extrapolate_flag = True, ssl_flag = False

In [ ]:
conv = "GAT"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ssl_flag = False
extrapolate_flag = True

    #######


optuna_training = TrainModelOptunaGC(
        data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
        )

best_values = optuna_training.run(number_of_trials=50)

model_training = TrainModelGC(
            data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
)

model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(best_values)
print(test_acc_mi)
results=results.append(pd.Series([extrapolate_flag,ssl_flag,test_acc_mi],index=results.columns), ignore_index=True)

## Extrapolate_flag = False, ssl=True


In [ ]:
conv = "GAT"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ssl_flag = True
extrapolate_flag = False

    #######
optuna_training = TrainModelOptunaGC(
        data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
        )

best_values = optuna_training.run(number_of_trials=50)

model_training = TrainModelGC(
            data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
)

model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(best_values)
print(test_acc_mi)
results=results.append(pd.Series([extrapolate_flag,ssl_flag,test_acc_mi],index=results.columns), ignore_index=True)

## Extrapolate_flag = True, ssl_flag = True

In [ ]:
conv = "GAT"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ssl_flag = True
extrapolate_flag = True

optuna_training = TrainModelOptunaGC(
        data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
        )

best_values = optuna_training.run(number_of_trials=50)

model_training = TrainModelGC(
            data=dataset,
            conv=conv,
            device=device,
            ssl_flag=ssl_flag,
            extrapolate_flag=extrapolate_flag,
)

model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(best_values)
print(test_acc_mi)
results=results.append(pd.Series([extrapolate_flag,ssl_flag,test_acc_mi],index=results.columns), ignore_index=True)

In [ ]:
results

## Проверим объяснение предсказания

In [ ]:
index = 3
data_to_explain = dataset[index]
root='../data_validation/'
if os.path.exists(root + name + "/A"+str(index)+".npy"):
    adj_matrix = np.load(root + name + "/A"+str(index)+".npy")
else:
    adj_matrix = torch.squeeze(to_dense_adj(data_to_explain.edge_index.cpu())).numpy()

if os.path.exists(root + name + "/X"+str(index)+".npy"):
    features = np.load(root + name + "/X"+str(index)+".npy")
else:
    features = torch.squeeze(data_to_explain.x.cpu()).numpy()


explainer = Explain(model=model, adj_matrix=adj_matrix, features=features)

pgm_explanation = explainer.structure_learning()
print("explanations is", pgm_explanation.nodes, pgm_explanation.edges)
g = nx.DiGraph()
mapping = {}
inv_mapping = {}
for i, node in enumerate(pgm_explanation.nodes):
    mapping[node]=i
    inv_mapping[i]=node

edges = []
for edge in pgm_explanation.edges:
    edges.append([mapping[edge[0]], mapping[edge[1]]])
g.add_edges_from(edges)
for node in g.nodes():
    print(node)
    g.add_node(node)

plt.figure()
nx.draw(g)
plt.title('data explanation' )
plt.show()
plt.close()